In [1]:
import importlib, os, gsw, gc

import pandas as pd
import xarray as xr
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import fmin
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
%matplotlib widget
import matplotlib.dates as mdates

import SteadyState2Constraints as ss2
import SXBQ as sx
importlib.reload(ss2)
importlib.reload(sx)

<module 'SXBQ' from 'C:\\Users\\johan\\OneDrive\\Bureau\\PFE-Goteborg\\Scripts\\SXBQ.py'>

In [2]:
top_mounted = False

#Useful functions
def rmsd(x):
    return np.sqrt(np.nanmean(x**2))
def smooth(x,N):
    return np.convolve(x, np.ones(N)/N, mode='same')
def fillGaps(x,y):
    f = interp1d(x[np.isfinite(x+y)],y[np.isfinite(x+y)], bounds_error=False, fill_value=np.NaN)
    return(f(x))

# LOAD PLD (PAYLOAD) AND NAV (NAVIGATION) DATA

In [3]:
data = sx.sxdf()
data.data = pd.read_parquet('C://Users/johan/OneDrive/Bureau/PFE-Goteborg/results_BT2.pqt')

def delimitateProfiles():
    #data.data.sort_values('Timestamp', ignore_index=True, inplace=True)
    data.median_resample()

    _gd = np.isfinite(data.data['diveNum'].values)
    _, _tmp = np.unique( np.round(data.data['diveNum'].values[_gd]) , return_inverse=True)

    data.data.loc[_gd,'diveNum'] = np.round(_tmp+1)
    data.data['diveNum'] = data.data['diveNum'].interpolate('nearest')
        
    data.data['profileNum'] = data.data['diveNum'].values*2
    _tmp = data.data['NAV_RESOURCE'].interpolate('nearest').values
    ind = (_tmp == 100) | (_tmp == 110) | (_tmp == 116)
    data.data.loc[ind,'profileNum'] = data.data.loc[ind,'profileNum'] - 1

delimitateProfiles()

def rmsd(x):
    return np.sqrt(np.nanmean(x**2))

#data.data

# LOAD ADCP DATA

In [4]:
#for first set of data
adcp_path = 'C://Users/johan/OneDrive/Bureau/PFE-Goteborg/Scripts/Data/SEA045_M44/ADCP/sea045_M44_A0'

header = ['Month','Day','Year','Hour','Minute','Second','Burst counter','Ensemble counter','Error code','Status code','Battery voltage','Soundspeed','Heading','Pitch','Roll','Pressure','Temperature','Analog input 1','Analog input 2']
ADCP = pd.read_table(adcp_path+'.sen',sep=r"\s+",names=header)
ADCP.insert(0,'time',pd.to_datetime(ADCP[['Year', 'Month', 'Day', 'Hour', 'Minute','Second']], utc=True, origin='unix', cache='False'))

ADCP.set_index('time', inplace=True)

ADCP = ADCP.to_xarray()

for beam in ['1','2','3','4']:
    ADCP['V'+beam] = (
                   ['time','bin'],
                   pd.read_csv(adcp_path+'.v'+beam, sep=r"\s+", header=None).iloc[:,2:]
                  )
    ADCP['C'+beam] = (
                   ['time','bin'],
                   pd.read_csv(adcp_path+'.c'+beam, sep=r"\s+", header=None).iloc[:,2:]
                  )

ADCP

<xarray.Dataset>
Dimensions:           (bin: 30, time: 277034)
Coordinates:
  * time              (time) object 1626942600438000000 ... 1629716499438000000
Dimensions without coordinates: bin
Data variables: (12/27)
    Month             (time) int64 7 7 7 7 7 7 7 7 7 7 7 ... 8 8 8 8 8 8 8 8 8 8
    Day               (time) int64 22 22 22 22 22 22 22 ... 23 23 23 23 23 23 23
    Year              (time) int64 2021 2021 2021 2021 ... 2021 2021 2021 2021
    Hour              (time) int64 8 8 8 8 8 8 8 8 8 ... 10 10 10 10 10 11 11 11
    Minute            (time) int64 30 30 30 30 30 30 31 31 ... 1 1 1 1 1 1 1 1
    Second            (time) float64 0.438 10.44 20.44 ... 19.44 29.44 39.44
    ...                ...
    V2                (time, bin) float64 -1.153 0.258 -0.54 ... -0.472 2.432
    C2                (time, bin) float64 2.0 2.0 0.0 2.0 ... 3.0 2.0 4.0 3.0
    V3                (time, bin) float64 -0.009 -0.509 -1.71 ... -1.826 -0.009
    C3                (time, bin) float64 38.0 6.0 6.0 2.0 ... 0.0 5.0 5.0 0.0
    V4                (time, bin) float64 -0.062 -1.481 0.917 ... -0.617 0.699
    C4                (time, bin) float64 21.0 2.0 0.0 4.0 ... 2.0 0.0 1.0 1.0

In [5]:
#------------------------------------------------
#------------------ BROUILLLON ------------------
#------------------------------------------------

za = np.asarray([ [1,1,1,1], [2,2,2,2], [4,4,4,4], [5,5,5,5], [8,8,8,8], [9,9,9,9]])
from scipy.signal import convolve2d

new_ntime_sample = 3 #dividing the number of sample relative to time 
new_nbin_sample = 2 #dividing the number of sample relative to space 
kernel = np.ones((new_ntime_sample, new_nbin_sample))
convolved = convolve2d(za, kernel, mode='valid')
print(convolved)
za_downsampled = convolved[::new_ntime_sample, ::new_nbin_sample]/(new_nbin_sample*new_ntime_sample)
print(za_downsampled)

[[14. 14. 14.]
 [22. 22. 22.]
 [34. 34. 34.]
 [44. 44. 44.]]
[[2.33333333 2.33333333]
 [7.33333333 7.33333333]]


# Magnetic deviation correction

In [6]:
# !!!!!!!! data['Heading'] contains a sample each 20s ==> prefer to use ADCP['Heading']

import geomag as geomag
def mag_dev_correction():
    ADCP_time = np.asarray(   ADCP.time.values.astype('float').copy()  /(10**9)   ) #ADCP time in second
    NAV_time = np.asarray(   sx.date2float(data.data['Timestamp'].copy())    ) #NAV time in second
    NAV_lat = data.data['lat'].values.copy()
    NAV_lon = data.data['lon'].values.copy()
    ADCP['lat'] = np.interp(ADCP_time, NAV_time, NAV_lat)
    ADCP['lon'] = np.interp(ADCP_time, NAV_time, NAV_lon)
    ADCP_lat = ADCP['lat'].values.copy()
    ADCP_lon = ADCP['lon'].values.copy()
    NAV_declination = np.zeros_like(NAV_time)
    ADCP_declination = np.zeros_like(ADCP_time)
    for i in tqdm(range(len(ADCP_time))) :
        ADCP_declination[i] = geomag.declination(ADCP_lat[i], ADCP_lon[i]) 
    ADCP['Heading'] = ADCP['Heading'].copy() + ADCP_declination
    NAV_declination = np.interp(NAV_time, ADCP_time, ADCP_declination)
    data.data['Heading'] = data.data['Heading'].values.copy() + NAV_declination
    
mag_dev_correction()

100%|████████████████████████████████████████████████████████████████████████| 277034/277034 [01:18<00:00, 3543.82it/s]


# LOAD BOTTOM TRACKING DATA

In [167]:
#for Bottom Tracking data
bt_path = 'C://Users/johan/OneDrive/Bureau/PFE-Goteborg/Scripts/Data/SEA045_M44/ADCP/telemetryfile'

header = ['Type','Beam','Date','Hour_Min_Sec','DT1','DT2','Bottom Velocity','Figure of Merit','Distance','Water Velocity','Status']
Telemetry = pd.read_table(bt_path+'.bin',sep=',',names=header) #To select only the first 1000 rows, use : nrows=1000
print("Data reading : OK")

BT = Telemetry[Telemetry['Type'] == '$PNORBT']      #Only keeps bottom tracking data from telemetry
BT = BT[BT['Distance'] > 0]                         #Only keep samples in range (distance not too far)
#BT = BT[BT['Distance'] < 100]                        #Only keep samples in range (distance not too close)
BT = BT[BT['Bottom Velocity'] < 2.5]                #Only keep speed that are in range 
BT = BT[BT['Bottom Velocity'] > -2.5]                #Only keep speed that are in range



#Set time as Index
BT = BT.astype({"Hour_Min_Sec": 'str', "Date": 'str'}) #Convert time and date as a string
BT['Hour_Min_Sec']=pd.to_datetime(BT['Hour_Min_Sec'], format="%H%M%S.%f", errors='coerce').dt.time #Convert to Pandas time format
BT['Date']=pd.to_datetime(BT['Date'], format="%m%d%y", errors='coerce') #Convert to Pandas date format
BT = BT[BT['Hour_Min_Sec'].notnull()] #Check for errors in time
BT = BT[BT['Date'].notnull()] #Check for errors in date
print("All NaT values have been deleted from date and time")
BT.insert(0,'time',pd.to_datetime(BT['Date'].astype(str) + ' ' + BT['Hour_Min_Sec'].astype(str), utc=True, origin='unix', cache='False'))
BT.set_index('time', inplace=True)
if (~BT.index.is_monotonic) :
    print("Some data were not sorted by date...")
    BT=BT.sort_index() #Some data were not sorted by date
    if (BT.index.is_monotonic) :
        print("     Data is now sorted by date")





#Put Beam 1-4 on a single row
BT_1 = BT[BT['Beam'] ==1]
BT_2 = BT[BT['Beam'] ==2]
BT_3 = BT[BT['Beam'] ==3]
BT_4 = BT[BT['Beam'] ==4]

if not(BT_1.index.is_unique and BT_2.index.is_unique and BT_3.index.is_unique and BT_4.index.is_unique) :
    print('Duplicate index detected...')
    BT_1 = BT_1[~BT_1.index.duplicated(keep='first')]
    BT_2 = BT_2[~BT_2.index.duplicated(keep='first')]
    BT_3 = BT_3[~BT_3.index.duplicated(keep='first')]
    BT_4 = BT_4[~BT_4.index.duplicated(keep='first')]
    print('     Duplicate index deleted')
    

BT = BT_1 #Put all 4 beam on one row

if not(BT_1.columns.is_unique and BT_2.columns.is_unique and BT_3.columns.is_unique and BT_4.columns.is_unique) :
    print('Duplicate column detected')

BT.insert(0,'V1',BT_1['Bottom Velocity'])
BT.insert(0,'D1',BT_1['Distance'])
BT.insert(0,'FoM1',BT_1['Figure of Merit'])

BT.insert(0,'V2',BT_2['Bottom Velocity'])
BT.insert(0,'D2',BT_2['Distance'])
BT.insert(0,'FoM2',BT_2['Figure of Merit'])

BT.insert(0,'V3',BT_3['Bottom Velocity'])
BT.insert(0,'D3',BT_3['Distance'])
BT.insert(0,'FoM3',BT_3['Figure of Merit'])

BT.insert(0,'V4',BT_4['Bottom Velocity'])
BT.insert(0,'D4',BT_4['Distance'])
BT.insert(0,'FoM4',BT_4['Figure of Merit'])

#print(BT_1['Bottom Velocity'])
#print(BT)

C:\Users\johan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Data reading : OK
All NaT values have been deleted from date and time
Some data were not sorted by date...
     Data is now sorted by date
Duplicate index detected...
     Duplicate index deleted


In [168]:
# Add roll pitch and heading to Bottom tracking data
A = BT.to_xarray()

#Insert Pitch
ADCP_Pitch = pd.DataFrame(   np.array([A['time'], np.interp(A['time'], ADCP['time'], ADCP['Pitch'])]).T , columns=['time', 'Pitch']  )
#print(ADCP_Pitch)
ADCP_Pitch['time']=pd.to_datetime(ADCP_Pitch['time']/(10**3), unit='us', utc=True, origin='unix', cache='False')
#print(ADCP_Pitch)
ADCP_Pitch.set_index('time', inplace=True)
#print(ADCP_Pitch)
BT.insert(0,'Pitch', ADCP_Pitch)

#Insert Roll
ADCP_Roll = pd.DataFrame(   np.array([A['time'], np.interp(A['time'], ADCP['time'], ADCP['Roll'])]).T , columns=['time', 'Roll']  )
#print(ADCP_Roll)
ADCP_Roll['time']=pd.to_datetime(ADCP_Roll['time']/(10**3), unit='us', utc=True, origin='unix', cache='False')
#print(ADCP_Roll)
ADCP_Roll.set_index('time', inplace=True)
#print(ADCP_Roll)
BT.insert(0,'Roll', ADCP_Roll)

#Insert Heading
ADCP_Heading = pd.DataFrame(   np.array([A['time'], np.interp(A['time'], ADCP['time'], ADCP['Heading'])]).T , columns=['time', 'Heading']  )
#print(ADCP_Heading)
ADCP_Heading['time']=pd.to_datetime(ADCP_Heading['time']/(10**3), unit='us', utc=True, origin='unix', cache='False')
#print(ADCP_Heading)
ADCP_Heading.set_index('time', inplace=True)
#print(ADCP_Heading)
BT.insert(0,'Heading', ADCP_Heading)

BT = BT.astype({"Pitch": 'float64', "Roll": 'float64', "Heading": 'float64',"Figure of Merit": 'float64'}) #Convert Pitch, Roll & Heading from string to float

BT = BT.to_xarray()
#print(BT)

# CALCULATE ADCP BIN DEPTH

In [9]:
def getADCPBinDepth(ADCP, bin_size, blanking_distance,direction='down',lat=58):
    # Retuns a new coordinate within the ADCP matrix of size ntime x nbin containing ADCP bin depths
    if top_mounted:
        direction = 1
    else:
        direction = -1
 
    #PREVIOUS CORRECTIONS (alpha here is angle between Zadcp and Z geographic)
    #bin_distance = blanking_distance + np.arange(len(ADCP.bin))*bin_size + 0.5*bin_size
    #alpha = np.arccos(
    #            np.cos(np.deg2rad(22.5 + ADCP['Pitch'])) * # THIS IS WRONG
    #            np.cos(np.deg2rad(ADCP['Roll'])) 
    #        )
    
    bin_distance = (blanking_distance + np.arange(len(ADCP.bin))*bin_size + 0.5*bin_size) *np.cos(np.deg2rad(22.5)) 
    alpha = np.arccos(
                np.cos(np.deg2rad(22.5))*np.cos(np.deg2rad(ADCP['Pitch']))*np.cos(np.deg2rad(ADCP['Roll']))
                + np.abs(   np.sin(np.deg2rad(22.5))*np.sin(np.deg2rad(ADCP['Pitch']))   )
            )
    
    adcp_depth = gsw.z_from_p(ADCP['Pressure'],lat)  #Depth is negatice underwater

    bin_depth = np.tile(adcp_depth, (len(ADCP.bin), 1)).T \
                + direction \
                * np.tile(bin_distance, (len(ADCP.time), 1)) \
                * np.tile(np.cos(alpha), (len(ADCP.bin), 1)).T \

    return ADCP.assign_coords({'bin_depth':(['time','bin'], bin_depth)})

ADCP = getADCPBinDepth(ADCP, 1, 0.2, direction='down', lat=58) #Or - in direction ?

# SOUNDSPEED CORRECTION

In [10]:
# Not implemented yet

# Rotation correction

In [11]:
def rot_correction ():
    ADCP['dPitch_dt'] = (['time'], np.gradient(np.deg2rad(ADCP.Pitch.values.astype('float')),ADCP.time.values.astype('float')/(10**9)))
    dx=0.25
    dz=0.1
    a=47.5
    b=25
    ADCP['V1'] = ADCP['V1'] + ADCP['dPitch_dt']*(-dz*np.sin(np.deg2rad(a))+dx*np.cos(np.deg2rad(a)))
    ADCP['V3'] = ADCP['V3'] + ADCP['dPitch_dt']*(dz*np.sin(np.deg2rad(a))+dx*np.cos(np.deg2rad(a)))
    ADCP['V2'] = ADCP['V2'] + ADCP['dPitch_dt']*( dx*np.cos(np.deg2rad(b)) )
    ADCP['V4'] = ADCP['V4'] + ADCP['dPitch_dt']*( dx*np.cos(np.deg2rad(b)) )

#rot_correction()

# Bottom Tracking (3 beam pitch-dependent solution to XYZ and ENU calculation)

In [171]:
def calcXYZfrom3beam():
    def sin(x):
        return np.sin(np.deg2rad(x))
    def cos(x):
        return np.cos(np.deg2rad(x))

    a = 47.5 # Beam 1 and 3 angle from Z
    b = 25 # Beam 2 and 4 angle from Z

    xyz2beam_fore = np.array([
        [sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])
    
    xyz2beam_aft = np.array([
        [-sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])

    beam2xyz_fore = np.linalg.inv(xyz2beam_fore)
    beam2xyz_aft = np.linalg.inv(xyz2beam_aft)


    V_fore = beam2xyz_fore @ np.array([
        BT['V1'].values.flatten(),
        BT['V2'].values.flatten(),
        BT['V4'].values.flatten()
        ])
    V_aft = beam2xyz_aft @ np.array([
        BT['V3'].values.flatten(),
        BT['V2'].values.flatten(),
        BT['V4'].values.flatten()
        ])

    if rmsd(V_aft[1:,:]-V_fore[1:,:]) != 0:
        print('Something is wrong - abort and investigate...')


    X_fore = np.reshape( V_fore[0,:] , (-1,1) ) #Because number of cell is 1
    X_aft = np.reshape( V_aft[0,:] , (-1,1) ) #Because number of cell is 1
    Y_fore = np.reshape( V_fore[1,:] , (-1,1) ) #Because number of cell is 1
    Y_aft = np.reshape( V_aft[1,:] , (-1,1) ) #Because number of cell is 1
    Z_fore = np.reshape( V_fore[2,:] , (-1,1) ) #Because number of cell is 1
    Z_aft = np.reshape( V_aft[2,:] , (-1,1) ) #Because number of cell is 1
    
    use_aft_on_climb = BT['Pitch'] > 0
    
    if top_mounted == True:
        print('Assuming ADCP is top mounted')
        X_fore[~use_aft_on_climb,:] = X_aft[~use_aft_on_climb,:]
        Y_fore[~use_aft_on_climb,:] = Y_aft[~use_aft_on_climb,:]
        Z_fore[~use_aft_on_climb,:] = Z_aft[~use_aft_on_climb,:]
    else:
        print('Assuming ADCP is bottom mounted')
        X_fore[use_aft_on_climb,:] = X_aft[use_aft_on_climb,:]
        Y_fore[use_aft_on_climb,:] = Y_aft[use_aft_on_climb,:]
        Z_fore[use_aft_on_climb,:] = Z_aft[use_aft_on_climb,:]
        Y_fore=-Y_fore
        Z_fore=-Z_fore

    BT['X'] = (['time','bin'], X_fore )
    BT['Y'] = (['time','bin'], Y_fore ) 
    BT['Z'] = (['time','bin'], Z_fore )
    
    plt.close('all')
    _ = plt.hist(X_fore.flatten(),100,color='r',alpha=0.3)
    _ = plt.hist(X_aft.flatten(),100,color='b',alpha=0.3)
    
    
calcXYZfrom3beam()
#gc.collect()

Assuming ADCP is bottom mounted


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [172]:
def calcENUfromXYZ():
    def M_xyz2enu(heading,pitch,roll):
        hh = np.pi*(heading-90)/180
        pp = np.pi*pitch/180
        rr = np.pi*roll/180
        
        _H = np.array([
            [np.cos(hh),np.sin(hh),0], 
            [-np.sin(hh),np.cos(hh),0], 
            [0,0,1]
        ])
        _P = np.array([
            [np.cos(pp), -np.sin(pp)*np.sin(rr), -np.cos(rr)*np.sin(pp)] ,
            [0, np.cos(rr), -np.sin(rr)] , 
            [ np.sin(pp), np.sin(rr)*np.cos(pp), np.cos(pp)*np.cos(rr)]
        ])
        
        _R = _H@_P
        return _R

    H = BT['Heading'].values
    P = BT['Pitch'].values
    R = BT['Roll'].values

    E = BT['X'].values.copy()
    N = BT['Y'].values.copy()
    U = BT['Z'].values.copy()


    r = len(E)

    for i in tqdm(range(r)):
        XYZ2ENU = M_xyz2enu(H[i],P[i],R[i])
        E[i], N[i], U[i] = XYZ2ENU @ [E[i], N[i], U[i]]

    BT['E'] =  (['time','bin'], E )
    BT['N'] =  (['time','bin'], N )
    BT['U'] =  (['time','bin'], U )

calcENUfromXYZ()
gc.collect()

100%|█████████████████████████████████████████████████████████████████████████| 33435/33435 [00:01<00:00, 29053.33it/s]


81

# ADCP (3 beam pitch-dependent solution to XYZ and ENU calculation)

In [14]:
def calcXYZfrom3beam_bis():
    def sin(x):
        return np.sin(np.deg2rad(x))
    def cos(x):
        return np.cos(np.deg2rad(x))

    a = 47.5 # Beam 1 and 3 angle from Z
    b = 25 # Beam 2 and 4 angle from Z

    xyz2beam_fore = np.array([
        [sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])
    
    xyz2beam_aft = np.array([
        [-sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])

    beam2xyz_fore = np.linalg.inv(xyz2beam_fore)
    beam2xyz_aft = np.linalg.inv(xyz2beam_aft)

    V_fore = beam2xyz_fore @ np.array([
        ADCP['V1'].values.flatten(),
        ADCP['V2'].values.flatten(),
        ADCP['V4'].values.flatten()
        ])
    V_aft = beam2xyz_aft @ np.array([
        ADCP['V3'].values.flatten(),
        ADCP['V2'].values.flatten(),
        ADCP['V4'].values.flatten()
        ])

    if rmsd(V_aft[1:,:]-V_fore[1:,:]) != 0:
        print('Something is wrong - abort and investigate...')

    X_fore = np.reshape( V_fore[0,:] , (-1,30) ) #Because number of cell is 30
    X_aft = np.reshape( V_aft[0,:] , (-1,30) ) #Because number of cell is 30
    Y_fore = np.reshape( V_fore[1,:] , (-1,30) ) #Because number of cell is 30
    Y_aft = np.reshape( V_aft[1,:] , (-1,30) ) #Because number of cell is 30
    Z_fore = np.reshape( V_fore[2,:] , (-1,30) ) #Because number of cell is 30
    Z_aft = np.reshape( V_aft[2,:] , (-1,30) ) #Because number of cell is 30
    
    
    use_aft_on_climb = ADCP['Pitch'] > 0
    
    if top_mounted == True:
        print('Assuming ADCP is top mounted')
        X_fore[~use_aft_on_climb,:] = X_aft[~use_aft_on_climb,:]
        Y_fore[~use_aft_on_climb,:] = Y_aft[~use_aft_on_climb,:]
        Z_fore[~use_aft_on_climb,:] = Z_aft[~use_aft_on_climb,:]
        
    else:
        print('Assuming ADCP is bottom mounted')
        X_fore[use_aft_on_climb,:] = X_aft[use_aft_on_climb,:]
        Y_fore[use_aft_on_climb,:] = Y_aft[use_aft_on_climb,:]
        Z_fore[use_aft_on_climb,:] = Z_aft[use_aft_on_climb,:]
        Y_fore=-Y_fore
        Z_fore=-Z_fore

    ADCP['X'] = (['time','bin'], X_fore )
    ADCP['Y'] = (['time','bin'], Y_fore ) 
    ADCP['Z'] = (['time','bin'], Z_fore ) 
    
    plt.close('all')
    _ = plt.hist(X_fore.flatten(),100,color='g',alpha=0.3)
    
calcXYZfrom3beam_bis()
#gc.collect()

Assuming ADCP is bottom mounted


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
def calcENUfromXYZ():
    def M_xyz2enu(heading,pitch,roll):
        hh = np.pi*(heading-90)/180
        pp = np.pi*pitch/180
        rr = np.pi*roll/180
        
        _H = np.array([
            [np.cos(hh),np.sin(hh),0], 
            [-np.sin(hh),np.cos(hh),0], 
            [0,0,1]
        ])
        _P = np.array([
            [np.cos(pp), -np.sin(pp)*np.sin(rr), -np.cos(rr)*np.sin(pp)] ,
            [0, np.cos(rr), -np.sin(rr)] , 
            [ np.sin(pp), np.sin(rr)*np.cos(pp), np.cos(pp)*np.cos(rr)]
        ])
        
        _R = _H@_P
        return _R

    H = ADCP['Heading'].values
    P = ADCP['Pitch'].values
    R = ADCP['Roll'].values

    E = ADCP['X'].values.copy()
    N = ADCP['Y'].values.copy()
    U = ADCP['Z'].values.copy()

    r,c = np.shape(E)

    for i in tqdm(range(r)):
        XYZ2ENU = M_xyz2enu(H[i],P[i],R[i])
        for j in range(c):
            E[i,j], N[i,j], U[i,j] = XYZ2ENU @ [E[i,j], N[i,j], U[i,j]]

    ADCP['E'] = (['time','bin'], E )
    ADCP['N'] = (['time','bin'], N )
    ADCP['U'] = (['time','bin'], U )

calcENUfromXYZ()
gc.collect()

100%|████████████████████████████████████████████████████████████████████████| 277034/277034 [00:39<00:00, 6928.34it/s]


35

In [18]:
plt.figure()

PD = ADCP['Pitch'] < 0
PU = ADCP['Pitch'] > 0

plt.subplot(511)
_ = plt.hist(ADCP.isel(bin=0)['X'][PD].values,np.linspace(-1,1,300),color='r')
_ = plt.hist(ADCP.isel(bin=0)['X'][PU].values,np.linspace(-1,1,300),color='b')
plt.title('Glider moving forward so expect X negative')

plt.subplot(513)
_ = plt.hist(ADCP.isel(bin=0)['U'][PD].values,np.linspace(-1,1,300),color='r')
plt.title('Glider diving so expect U positive')

plt.subplot(515)
_ = plt.hist(ADCP.isel(bin=0)['U'][PU].values,np.linspace(-1,1,300),color='b')
plt.title('Glider climbing so expect U negative')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Glider climbing so expect U negative')

# ------------------------ LADCP ------------------------

# Baroclinic component

In [17]:
#Create array of dives number (dive == between two resurface)

ADCP_time = np.asarray(   ADCP.time.values.astype('float')/(10**9)   ) #ADCP time in second
NAV_time = np.asarray(sx.date2float(data.data['Timestamp'])) #NAV time in second

#Glider depth
ADCP_depth = (data.data['pressure']).copy().ffill().bfill()
ADCP_depth = ADCP_depth.values.astype('int')
ADCP_depth = np.asarray(   gsw.z_from_p(ADCP_depth, lat=58)   )
ADCP_depth = np.floor(   np.interp(ADCP_time, NAV_time, ADCP_depth)   ).astype('int') 


#Glider situation : 116==transmitting // 115==surfacing
ADCP_Situation = (data.data['NAV_RESOURCE']).copy().ffill().bfill()
ADCP_Situation = (ADCP_Situation.values.astype('int'))
ADCP_Situation[ADCP_Situation!=116] = 115
ADCP_Situation= np.floor((smooth(ADCP_Situation,1))).astype('int') #This line is useless
ADCP_Situation = np.roll(ADCP_Situation, 300)
ADCP_Situation = np.floor(   np.interp(ADCP_time, NAV_time, ADCP_Situation)   ).astype('int') 

#ADCP_Situation[(ADCP_Situation!=116) & (ADCP_Situation!=115)]=0
#ADCP_Situation[ADCP_Situation==116]=1
#ADCP_Situation[ADCP_Situation==115]=1
#ADCP_Situation=ADCP_Situation.astype('int')

#Dive number (nth dive) of the glider
ADCP_DiveNumber = np.zeros_like(ADCP_depth,dtype='int')
ADCP_DiveNumber[ (ADCP_depth>-5) & (ADCP_Situation==116)]=1 #We separate each dive when the drone comes at depth = -3m
ADCP_DiveNumber = np.diff(ADCP_DiveNumber,prepend=1)
ADCP_DiveNumber[ADCP_DiveNumber==-1] = 0 #------------- !!!!! =1 ou bien =0


ADCP_DiveNumber_time=ADCP_time[ADCP_DiveNumber==1] #Start time of each flight (first flight not taken into account)
ADCP_DiveNumber_time = np.insert(ADCP_DiveNumber_time, 0, ADCP_time[0])#Adding time of start of first flight

ADCP_DiveNumber = np.cumsum(ADCP_DiveNumber)

#print(ADCP_DiveNumber[4900:5500])
print(len(ADCP_DiveNumber_time))
print(  (ADCP_DiveNumber[-1]+1)  )

217
217


In [19]:
#Calculate shear profile for each ADCP sample (1 each 10 seconds) :

print(len(ADCP['E']))
print(len(ADCP['E'][0]))
print(len(ADCP_DiveNumber))

ADCP['Shear_E'] = (['time','bin'], ADCP['E'].values.copy() )
ADCP['Shear_N'] = (['time','bin'], ADCP['N'].values.copy() )

for t in tqdm(range(len(ADCP['E']))):
    ADCP['Shear_E'][t]=np.gradient(ADCP['E'][t],ADCP['bin_depth'][t])
    ADCP['Shear_N'][t]=np.gradient(ADCP['N'][t],ADCP['bin_depth'][t])


#Delete values when at the surface and during inflection points : (optional)

#(ADCP['Shear_E'])[np.abs(ADCP['Pitch'])<0.2]=np.nan #Optional
#(ADCP['Shear_N'])[np.abs(ADCP['Pitch'])<0.2]=np.nan #Optional
#(ADCP['Shear_E'])[ ADCP_depth>-5 ]=np.nan #Optional
#(ADCP['Shear_N'])[ ADCP_depth>-5 ]=np.nan #Optional

  0%|                                                                            | 115/277034 [00:00<08:04, 571.52it/s]

277034
30
277034


100%|█████████████████████████████████████████████████████████████████████████| 277034/277034 [07:24<00:00, 623.19it/s]


In [21]:
#-----------------------------------------
#--------------- BROUILLON ---------------
#-----------------------------------------

t_start = ADCP_DiveNumber_time[10]
t_end = ADCP_DiveNumber_time[10+1]
print(t_start)
print(t_end)
print([ (ADCP_time>t_start) & (ADCP_time<t_end) ])
Shear_samples_to_consider=ADCP_time[ (ADCP_time>t_start) & (ADCP_time<t_end) ]

print(Shear_samples_to_consider)

1627013110.4380002
1627026440.4380002
[array([False, False, False, ..., False, False, False])]
[1.62701312e+09 1.62701313e+09 1.62701314e+09 ... 1.62702641e+09
 1.62702642e+09 1.62702643e+09]


In [56]:
#Construct a map of shear with respect to depth 

nDive = (ADCP_DiveNumber[-1]+1) #Number of dive for the deployment, first dive : nDive=0
dz=1.0 #must be close to the cell size of the ADCP[m]
H=450 #[m] take H larger than what the ADCP can reach during the dive (larger than MAx Depth + ADCP range)
Nz=(np.floor(H/dz)).astype('int')

#Shear_East=np.zeros((nDive,Nz))
Shear_East=np.empty((nDive,Nz))
Shear_East[:] = np.nan
Shear_North=np.empty((nDive,Nz))
Shear_North[:] = np.nan




#############
#Std_East=np.empty((nDive,Nz))
#Std_East[:] = np.nan
#Std_North=np.empty((nDive,Nz))
#Std_North[:] = np.nan
##############




print(Shear_East[10,53]) #This is Shear for dive number 10 and depth 53*dz=53*2=106m

for d in range(ADCP_DiveNumber[-1]+1):
    #Time interval to consider
    t_start = ADCP_DiveNumber_time[d]
    if (d==ADCP_DiveNumber[-1]):
        t_end = ADCP_time[-1]
    else :
        t_end = ADCP_DiveNumber_time[d+1]
        
    #Depth of the glider during the dive number d
    Depth_to_consider=( gsw.z_from_p(ADCP['Pressure'].values.copy(),lat=55) )[ (ADCP_time>t_start) & (ADCP_time<t_end) ]
    if len(Depth_to_consider)==0 :
        Max_depth=0
    else:
        Max_depth = np.nanmin(Depth_to_consider)
    nz_max_depth = (np.floor(-Max_depth/dz)).astype('int')
    
    #Shear samples to consider
    East_Shear_samples_to_consider=(ADCP['Shear_E'])[ (ADCP_time>t_start) & (ADCP_time<t_end) ]
    East_Shear_samples_to_consider=East_Shear_samples_to_consider.values.ravel()
    
    North_Shear_samples_to_consider=(ADCP['Shear_N'])[ (ADCP_time>t_start) & (ADCP_time<t_end) ]
    North_Shear_samples_to_consider=North_Shear_samples_to_consider.values.ravel()
    
    #Bin depth samples to consider
    Bin_depth_samples_to_consider=(ADCP['bin_depth'])[ (ADCP_time>t_start) & (ADCP_time<t_end) ]
    Bin_depth_samples_to_consider=Bin_depth_samples_to_consider.values.ravel()
    Bin_depth_samples_to_consider=(np.floor(-Bin_depth_samples_to_consider/dz)).astype('int')

    
    #Using dataframe to calculate the average shear for each depth dz
    df = pd.DataFrame({'nz': Bin_depth_samples_to_consider,
                   'Shear_E': East_Shear_samples_to_consider,
                   'Shear_N': North_Shear_samples_to_consider})
    
    #In order to have standard deviation, we must use .groupby().agg(), as follow :
    #df = df.groupby(['nz']).agg({'Shear_E':['mean','std'],'Shear_N':['mean','std']})   
    #df.columns = ['nz','Shear_E','Std_E','Shear_N','Std_N']
    
    df = df.groupby(['nz']).mean()
    #Delete samples that are "above the water" or "under the ground"
    if top_mounted :
        S_E=(df['Shear_E'])[df.index>=0]
        S_N=(df['Shear_N'])[df.index>=0]
    else :      
        S_E=(df['Shear_E'])[df.index<=nz_max_depth]
        S_N=(df['Shear_N'])[df.index<=nz_max_depth]
    (Shear_East[d])[0:len(S_E)]=S_E[0:len(S_E)]
    (Shear_North[d])[0:len(S_N)]=S_N[0:len(S_N)]

nan


In [57]:
%matplotlib widget
plt.close('all')
ndive_to_show=118

fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
X=Shear_East[ndive_to_show]
Y=np.linspace(-1, -H-1, num=Nz)
plt.plot(X,Y,'.')
plt.plot(X,Y)
plt.grid()
plt.xlabel('East component of shear [m/s /m]')
plt.ylabel('Depth [m]')
plt.title('Dive number : ' + str(ndive_to_show))


ax2 = fig.add_subplot(1, 2, 2)
X=Shear_North[ndive_to_show]
Y=np.linspace(-1, -H-1, num=Nz)
plt.plot(X,Y,'.')
plt.plot(X,Y)
plt.grid()
plt.xlabel('North component of shear [m/s /m]')
plt.ylabel('Depth [m]')
plt.title('Dive number : ' + str(ndive_to_show))

plt.tight_layout()
fig.canvas.draw_idle()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
#Compute Baroclinic speed :
Baroclinic_East=np.empty((nDive,Nz))
Baroclinic_North=np.empty((nDive,Nz))

for d_n in range(len(Baroclinic_East)) :
    for n_z in range(len(Baroclinic_East[0])) :
        if np.isnan(Shear_East[d_n,n_z]) :
            Baroclinic_East[d_n,n_z] = np.nan
        else :
            S_E=(Shear_East[d_n])[0:n_z]
            S_E=S_E[~np.isnan(S_E)] #Delete NaN values at the surface and at the bottom
            Integral_East=np.trapz(  y=S_E , dx=-dz) #We integrate from the surface to the bottom, so negative
            Baroclinic_East[d_n,n_z] = Integral_East
        
        if np.isnan(Shear_East[d_n,n_z]) :
            Baroclinic_North[d_n,n_z] = np.nan
        else :
            S_N=(Shear_North[d_n])[0:n_z]
            S_N=S_N[~np.isnan(S_N)] #Delete NaN values at the surface and at the bottom
            Integral_North=np.trapz(  y=S_N  , dx=-dz) #We integrate from the surface to the bottom, so negative 
            Baroclinic_North[d_n,n_z] = Integral_North
            
    Baroclinic_East[d_n]=Baroclinic_East[d_n]-np.nanmean(Baroclinic_East[d_n])
    Baroclinic_North[d_n]=Baroclinic_North[d_n]-np.nanmean(Baroclinic_North[d_n])

<ipython-input-58-3d69bf374af5>:23: RuntimeWarning: Mean of empty slice
  Baroclinic_East[d_n]=Baroclinic_East[d_n]-np.nanmean(Baroclinic_East[d_n])
<ipython-input-58-3d69bf374af5>:24: RuntimeWarning: Mean of empty slice
  Baroclinic_North[d_n]=Baroclinic_North[d_n]-np.nanmean(Baroclinic_North[d_n])


In [59]:
from mpl_toolkits.mplot3d import axes3d
from matplotlib.widgets import Slider, Button




%matplotlib widget
plt.close('all')
ndive_to_show=120

fig = plt.figure()

ax = fig.add_subplot(1, 2, 1)
X1=Baroclinic_East[ndive_to_show]
Y=np.linspace(-1, -H-1, num=Nz)
dot_East, = plt.plot(X1,Y,'.')
line_East, = plt.plot(X1,Y)
plt.xlim([-0.7, 0.7])
plt.ylim([0, -380])
plt.gca().invert_yaxis()
plt.grid()
plt.xlabel('East baroclinic speed [m/s]')
plt.ylabel('Depth')

ax2 = fig.add_subplot(1, 2, 2)
X2=Baroclinic_North[ndive_to_show]
Y=np.linspace(-1, -H-1, num=Nz)
dot_North, = plt.plot(X2,Y,'.')
line_North, = plt.plot(X2,Y)
plt.xlim([-0.7, 0.7])
plt.ylim([0, -380])
plt.gca().invert_yaxis()
plt.grid()
plt.xlabel('North baroclinic speed [m/s]')
plt.ylabel('Depth')
plt.tight_layout()


plt.subplots_adjust(left=0.25, bottom=0.25)
axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], facecolor=axcolor)
nDive_slider = Slider(
    ax=axfreq,
    label='Dive number',
    valstep=1,
    valmin=ADCP_DiveNumber[0],
    valmax=ADCP_DiveNumber[-1],
    valinit=np.floor(ADCP_DiveNumber[-1]/2),
)


def update(val):
    line_East.set_xdata(  Baroclinic_East[ nDive_slider.val]  )
    dot_East.set_xdata(  Baroclinic_East[ nDive_slider.val]  )
    line_North.set_xdata(  Baroclinic_North[ nDive_slider.val]  )
    dot_North.set_xdata(  Baroclinic_North[ nDive_slider.val]  )
    fig.canvas.draw_idle()

nDive_slider.on_changed(update)









fig2 = plt.figure()
ax3 = fig2.gca(projection='3d')
zline = np.linspace(-1, -H-1, num=Nz)
ax3.set_xlim3d(-0.5, 0.5)
ax3.set_ylim3d(-0.5, 0.5)
ax3.set_zlim3d(0, -140/100)
ax3.set_xlabel('East speed [m/s]')
ax3.set_ylabel('North speed [m/s]')
ax3.set_zlabel('Depth [m]')
ax3.set_box_aspect((1, 1, 1))
arrow_length_ratio=0.1
ax3.plot3D(zline*0, zline*0, zline/100, 'gray')
ax3.quiver(zline*0, zline*0, zline/100, X1, X2, zline*0)
ax3.quiver([0.4], [-0.4], [-140/100], [0], [0.5], [0], color='r', pivot='tail', length=1, arrow_length_ratio=0.3)
plt.gca().invert_zaxis()
plt.tight_layout()



fig.canvas.draw_idle()
fig2.canvas.draw_idle()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
#-----------------------------------------
#--------------- BROUILLON ---------------
#-----------------------------------------

nDive = (ADCP_DiveNumber[-1]+1) #Number of dive for the deployment, first dive : nDive=0
dz=1.0 #[m]
H=450.0 #[m] take H larger than what the ADCP can reach during the dive
Nz=(np.floor(H/dz)).astype('int')
Shear_East=np.zeros((nDive,Nz))

A=(ADCP['bin_depth'])[10000:10900]
A=A.values.ravel()
#print(A)
A=(np.floor(-A/2)).astype('int')
#print(A)
B=(ADCP['Shear_E'][10000:10900])
B=B.values.ravel()
#print(B)


df = pd.DataFrame({'nz': A,
                   'Shear_E': B})

df = df.groupby(['nz']).mean()

C=(df['Shear_E'])[df.index>=0]
print(len(C))
print(C[0:len(C)])
print(C)

# Steady State with 2nd Speed constraint

In [173]:
#Useful variables ----------------------------------------------------------------------------
time=sx.date2float(data.data['Timestamp'])

_u = np.remainder(data.data.profileNum,2) == 0
_d = np.remainder(data.data.profileNum,2) == 1

Temperature=fillGaps(time, data.data.Temperature)
pressure=fillGaps(time, data.data.pressure)
salinity=fillGaps(time, data.data.salinity)
Ballast=fillGaps(time, data.data.BallastPos/1000000)
lon=fillGaps(time, data.data.lon)
lat=fillGaps(time, data.data.lat)

Depth = gsw.z_from_p(pressure,lat) # m . Note depth (Z) is negative, so diving is negative dZdt      
dZdt =fillGaps(time, np.gradient(Depth,time))

SA = gsw.SA_from_SP(salinity, pressure, lon, lat)
CT = gsw.CT_from_t(SA, Temperature, pressure)
rho = gsw.rho(SA, CT, pressure)



# Interpolation of ADCP data in order to match with drone data (data.data) -----------------------------------------
smooth_coeff=50
ADCP_Time=ADCP.time.values.astype('float')/(10**9)

ADCP_spd_xyz = np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Y']**2 + ADCP.isel(bin=0)['Z']**2)
ADCP_speed=np.interp(time, ADCP_Time, ADCP_spd_xyz)

ADCP_xz_spd=np.interp(time, ADCP_Time, np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Z']**2) )

ADCP_vert_spd = np.interp(time, ADCP_Time, ADCP.isel(bin=0)['U'])

ADCP_x_spd=np.interp(time, ADCP_Time, ADCP.isel(bin=0)['X'])
ADCP_y_spd=np.interp(time, ADCP_Time, ADCP.isel(bin=0)['Y'])
ADCP_z_spd=np.interp(time, ADCP_Time, ADCP.isel(bin=0)['Z'])
ADCP_pitch=np.deg2rad(np.interp(time, ADCP_Time, ADCP.isel(bin=0)['Pitch']))
ADCP_hrz_spd =  (ADCP_x_spd)*np.cos(np.deg2rad(ADCP_pitch)) -(ADCP_z_spd)*np.sin(np.deg2rad(ADCP_pitch))

ADCP_pressure=np.interp(time, ADCP_Time, ADCP.Pressure.values.astype('float') )
ADCP_pitch=np.interp(time, ADCP_Time, ADCP.Pitch.values.astype('float') )
ADCP_roll=np.interp(time, ADCP_Time, ADCP.Roll.values.astype('float') )
ADCP_heading=np.interp(time, ADCP_Time, ADCP.Heading.values.astype('float') )



#Influence of drone rotation in ADCP measurement -------------------------------------------------------------------
omega = fillGaps(time, np.gradient(ADCP_pitch,time))

# Interpolation of Bottom Tracking data in order to match with drone data (data.data) --------------------------------------
BT_spd_xyz = np.sqrt(BT.isel(bin=0)['X']**2 + BT.isel(bin=0)['Y']**2 + BT.isel(bin=0)['Z']**2)
BT_Time=BT.time.values.astype('float')/(10**9)
#BT_spd_xyz = fillGaps(BT_spd_xyz,BT_Time)
BT_speed = np.interp(time, BT_Time, BT_spd_xyz)

BT_spd_xz = np.sqrt(BT.isel(bin=0)['X']**2 + BT.isel(bin=0)['Z']**2)
BT_xz_spd=np.interp(time, BT_Time, BT_spd_xz)

BT_vert_spd = np.interp(time, BT_Time, BT.isel(bin=0)['U'])

BT_x_spd=np.interp(time, BT_Time, BT.isel(bin=0)['X'])
BT_y_spd=np.interp(time, BT_Time, BT.isel(bin=0)['Y'])
BT_z_spd=np.interp(time, BT_Time, BT.isel(bin=0)['Z'])
BT_pitch=np.interp(time, BT_Time, BT['Pitch'])
BT_hrz_spd =  (BT_x_spd)*np.cos(np.deg2rad(BT_pitch)) - (BT_z_spd)*np.sin(np.deg2rad(BT_pitch))

In [61]:
importlib.reload(ss2)

ADCP_time = np.asarray(ADCP.time.values.astype('float')/(10**9)) #ADCP time in second
NAV_time = np.asarray(sx.date2float(data.data['Timestamp'])) #NAV time in second

ADCP_pitch = np.interp(NAV_time, ADCP_time, ADCP.Pitch.values.astype('float') ) #Pitch in degrees

flight = ss2.SteadyState_2_Model(
               data.data.Timestamp, 
               data.data.salinity.interpolate('index').values, 
               data.data.temperature.interpolate('index').values, 
               data.data.pressure.interpolate('index').values, 
               data.data.lon.interpolate('index').values, 
               data.data.lat.interpolate('index').values, 
               data.data.BallastPos.interpolate('index').values, 
               ADCP_pitch, 
               data.data.profileNum.interpolate('nearest').values,
               data.data.NAV_RESOURCE.interpolate('nearest').values,
               0.5, #Tau
               smooth(ADCP_speed, 50),
               mass=60.772) # Verify mass number !!

C:\Users\johan\OneDrive\Bureau\PFE-Goteborg\Scripts\SteadyState2Constraints.py:306: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(2 * _dynamic_pressure / self.rho / self.area_w)


In [62]:
flight.model_function()

In [63]:
flight.Apply_lowpass_filter()

In [ ]:
flight.regress()

In [64]:
#Compute the East and North components of the glider speed relative to water
ADCP_heading = np.interp(NAV_time, ADCP_time, ADCP.Heading.values.astype('float') ) #Heading in degrees

Glider_east_speed = (  flight.speed_horz_filtered.copy()  )*np.sin(  ADCP_heading*np.pi/180  ) #Heading negative is going west
Glider_north_speed = (  flight.speed_horz_filtered.copy()  )*np.cos(  ADCP_heading*np.pi/180  )
Glider_up_speed = flight.speed_vert_filtered.copy()

In [65]:
#Compute nz number and flight number of the glider for NAV data (index first positive)

NAV_FlightNumber = (  np.floor(  np.interp(NAV_time, ADCP_time, ADCP_DiveNumber)  )  ).astype('int')
NAV_depth = np.asarray( flight.depth.copy() )
NAV_nz = (np.floor(-NAV_depth/dz))
#NaN values are considered to be at the surface, so nz=0
NAV_nz[np.isnan(NAV_nz)] = 0
NAV_nz = NAV_nz.astype('int')
NAV_nz[NAV_nz<0]=0
print(NAV_nz[~np.isnan(NAV_nz)])

[ 0  0  0 ... 48 48 48]


# Barotropic component

In [ ]:
#-----------------------------------------
#--------------- BROUILLON ---------------
#-----------------------------------------

#Interpolate GPS Data so that error is not to large when choosingthe points 

#  len(data)=2 762 003

GPS_Depth = -(data.data['Depth']).copy().ffill().bfill()
GPS_Depth = GPS_Depth.values.astype('int')
GPS_Situation = (data.data['NAV_RESOURCE']).copy().ffill().bfill()
GPS_Situation = (GPS_Situation.values.astype('int'))
GPS_Situation[GPS_Situation!=116] = 115
A=GPS_Situation.copy()
GPS_Situation= np.floor((smooth(GPS_Situation,1))).astype('int')
GPS_Situation = np.roll(GPS_Situation, 300)
Y1_lat=data.data['lat'].copy()
Y2_lat=np.zeros_like(Y1_lat)
Y2_lat[(GPS_Situation==116)&(GPS_Depth>-5)]=1
Y3_lat = Y1_lat.copy()
Y3_lat[:] = np.nan
Y3_lat[Y2_lat==1] = Y1_lat[Y2_lat==1]
Y4_lat=(Y3_lat.copy()).interpolate(method='linear', axis=0).ffill().bfill()





Y5_lat= GPS_Situation
Y6_lat = GPS_Depth

A=A[100000:460000]
Y1_lat=Y1_lat[100000:460000]
Y2_lat=Y2_lat[100000:460000]
Y3_lat=Y3_lat[100000:460000]
Y4_lat=Y4_lat[100000:460000]
Y5_lat=Y5_lat[100000:460000]
Y6_lat=Y6_lat[100000:460000]
X=np.linspace(0,len(Y1_lat), len(Y1_lat))

%matplotlib widget
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(4, 1, 1)
plt.scatter(X,Y1_lat,1, label='data.data[lat]')
plt.plot(X,Y3_lat, color='r', label='GPS points to consider')
plt.scatter(X,Y4_lat,1,color='g',label='Y4_lat')
#ax.set_ylim([-1, 1.5])
ax.set_xlim([X[0], X[-1]])
plt.legend()

ax2 = fig.add_subplot(4, 1, 2)
plt.plot(X,Y5_lat)
ax2.set_ylim([112, 120])
ax2.set_xlim([X[0], X[-1]])

ax3 = fig.add_subplot(4, 1, 3)
plt.scatter(X,Y6_lat,1)
ax3.set_ylim([-30, 5])
ax3.set_xlim([X[0], X[-1]])

ax4 = fig.add_subplot(4, 1, 4)
plt.scatter(X,A,1)
plt.scatter(X,Y5_lat,1)
#ax3.set_ylim([-30, 5])
ax4.set_xlim([X[0], X[-1]])


fig.canvas.draw_idle()
plt.show()

In [66]:
#Compute average over-ground speed of the glider from GPS data
from geopy import distance


#Mise en forme et correction des données GPS

GPS_Depth = -(data.data['Depth']).copy().ffill().bfill()
GPS_Depth = GPS_Depth.values.astype('int')
GPS_Situation = (data.data['NAV_RESOURCE']).copy().ffill().bfill()
GPS_Situation = (GPS_Situation.values.astype('int'))
GPS_Situation[GPS_Situation!=116] = 115
GPS_Situation= np.floor((smooth(GPS_Situation,1))).astype('int') #This line is useless
GPS_Situation = np.roll(GPS_Situation, 300)

Y1_lat=data.data['lat'].copy()
Y2_lat=np.zeros_like(Y1_lat)
Y2_lat[(GPS_Situation==116)&(GPS_Depth>-5)]=1
Y3_lat = Y1_lat.copy()
Y3_lat[:] = np.nan
Y3_lat[Y2_lat==1] = Y1_lat[Y2_lat==1]
Y4_lat=(Y3_lat.copy()).interpolate(method='linear', axis=0).ffill().bfill()

Y1_lon=data.data['lon'].copy()
Y2_lon=np.zeros_like(Y1_lon)
Y2_lon[(GPS_Situation==116)&(GPS_Depth>-5)]=1
Y3_lon = Y1_lon.copy()
Y3_lon[:] = np.nan
Y3_lon[Y2_lon==1] = Y1_lon[Y2_lon==1]
Y4_lon=(Y3_lon.copy()).interpolate(method='linear', axis=0).ffill().bfill()


NAV_latitude = Y4_lat #Before : =data.data['lat'].copy()
NAV_longitude = Y4_lon #data.data['lon'].copy()




#Compute average over-ground speed of the glider
GPS_east_speed=np.zeros_like(ADCP_DiveNumber_time)
GPS_north_speed=np.zeros_like(ADCP_DiveNumber_time)

for d in range(len(GPS_east_speed)): 
    #Time interval to consider
    t_start = ADCP_DiveNumber_time[d]
    if (d==ADCP_DiveNumber[-1]):
        t_end = ADCP_time[-1]
    else :
        t_end = ADCP_DiveNumber_time[d+1]
    
    #Getting START and END GPS positions
    Start_latitude=np.interp(t_start, NAV_time, NAV_latitude)
    Start_longitude=np.interp(t_start, NAV_time, NAV_longitude)
    End_latitude=np.interp(t_end, NAV_time, NAV_latitude)
    End_longitude=np.interp(t_end, NAV_time, NAV_longitude)
    
    #computing relative distances (Eastward and Northward)
    latitude_distance=distance.distance(  (Start_latitude,Start_longitude)  ,  (End_latitude,Start_longitude)  ).m #Distance in meters
    if (End_latitude<Start_latitude) :
        latitude_distance = -latitude_distance
        
    longitude_distance=distance.distance(  (Start_latitude,Start_longitude)  ,  (Start_latitude,End_longitude)  ).m #Distance in meters
    if (End_longitude<Start_longitude) :
        longitude_distance = -longitude_distance
        
    #Computing Eastward and Northward speed
    delta_T = t_end - t_start
    GPS_east_speed[d] = longitude_distance/delta_T
    GPS_north_speed[d] = latitude_distance/delta_T

In [67]:
#Compute barotropic speed

#Compute "instantaneous" barotropic speed (no physical meaning, only used for computing purpose)
Ubarotrop_east_instantaneous = np.zeros_like(NAV_time)
for i in range(len( Ubarotrop_east_instantaneous )) :
    flight_number=NAV_FlightNumber[i]
    nz=NAV_nz[i]
    Ubarotrop_east_instantaneous[i] = GPS_east_speed[flight_number] - Glider_east_speed[i] - Baroclinic_East[flight_number,nz]
    
Ubarotrop_north_instantaneous = np.zeros_like(NAV_time)
for i in range(len( Ubarotrop_north_instantaneous )) :
    flight_number=NAV_FlightNumber[i]
    nz=NAV_nz[i]
    Ubarotrop_north_instantaneous[i] = GPS_north_speed[flight_number] - Glider_north_speed[i] - Baroclinic_North[flight_number,nz]

#True barotropic speed
Barotropic_East=np.zeros(nDive)
Barotropic_North=np.zeros(nDive)    
    
for d in range(ADCP_DiveNumber[-1]+1):
    #Time interval to consider
    t_start = ADCP_DiveNumber_time[d]
    if (d==ADCP_DiveNumber[-1]):
        t_end = ADCP_time[-1]
    else :
        t_end = ADCP_DiveNumber_time[d+1] 
    delta_T = t_end-t_start
    dt = 1.0
        
    #Instantaneous barotropic speed samples to consider
    Ubarotrop_east_inst_to_consider = Ubarotrop_east_instantaneous[ (NAV_time>t_start) & (NAV_time<t_end) ]
    Ubarotrop_north_inst_to_consider = Ubarotrop_north_instantaneous[ (NAV_time>t_start) & (NAV_time<t_end) ]

    #Barotropic speed for flight number d
    Barotropic_East[d] = np.nanmean(Ubarotrop_east_inst_to_consider)
    Barotropic_North[d] = np.nanmean(Ubarotrop_north_inst_to_consider)

<ipython-input-67-c679d5e8d526>:35: RuntimeWarning: Mean of empty slice
  Barotropic_East[d] = np.nanmean(Ubarotrop_east_inst_to_consider)
<ipython-input-67-c679d5e8d526>:36: RuntimeWarning: Mean of empty slice
  Barotropic_North[d] = np.nanmean(Ubarotrop_north_inst_to_consider)


In [ ]:
#-----------------------------------------
#--------------- BROUILLON ---------------
#-----------------------------------------

print(np.abs(Barotropic_East)>1)

In [ ]:
#-----------------------------------------
#--------------- BROUILLON ---------------
#-----------------------------------------

t_start = ADCP_DiveNumber_time[nDive_slider.val]
if (nDive_slider.val==ADCP_DiveNumber[-1]):
    t_end = ADCP_time[-1]
else :
    t_end = ADCP_DiveNumber_time[nDive_slider.val+1]
date_start=datetime.datetime.utcfromtimestamp(t_start)
print( type(date_start.strftime('%d %B %Y')))

In [68]:
import datetime
%matplotlib widget
plt.close('all')




ndive_to_show=100

H=450 #[m] take H larger than what the ADCP can reach during the dive
Nz=(np.floor(H/dz)).astype('int')

T=ADCP_DiveNumber_time.copy()
X_East=Baroclinic_East .copy()
X_North=Baroclinic_North.copy()

print(len(X_East))
print(len(X_North))
Selection=[~np.isnan(X_East[:,10])]
T=T[Selection]
X_East=X_East[Selection]
X_North=X_North[Selection]

X_East[:,0::3]=np.nan #Take only 1 each 3 speed for clearer visualization
X_North[:,0::3]=np.nan
X_East[:,1::3]=np.nan 
X_North[:,1::3]=np.nan

X1= X_East[ndive_to_show] + Barotropic_East[ndive_to_show]
X2 = X_North[ndive_to_show] + Barotropic_North[ndive_to_show]

Y=np.linspace(-1, -H-1, num=Nz)



fig = plt.figure()

ax = fig.add_subplot(1, 2, 1)
dot_East, = plt.plot(X1,Y,'.')
line_East, = plt.plot(X1,Y)
plt.xlim([-1, 1])
plt.ylim([0, -380])
plt.gca().invert_yaxis()
plt.grid()
plt.xlabel('East baroclinic speed [m/s]')
plt.ylabel('Depth')

ax2 = fig.add_subplot(1, 2, 2)
dot_North, = plt.plot(X2,Y,'.')
line_North, = plt.plot(X2,Y)
plt.xlim([-1, 1])
plt.ylim([0, -380])
plt.gca().invert_yaxis()
plt.grid()
plt.xlabel('North baroclinic speed [m/s]')
plt.ylabel('Depth')
plt.tight_layout()


plt.subplots_adjust(left=0.25, bottom=0.25)
axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], facecolor=axcolor)
nDive_slider = Slider(
    ax=axfreq,
    label='Dive number',
    valstep=1,
    valmin=0,
    valmax=len(X_East)-1,
    valinit=np.floor((len(X_East)-1)/2),
)


def update(val):
    X1= X_East[nDive_slider.val] + Barotropic_East[nDive_slider.val]
    X2 = X_North[nDive_slider.val] + Barotropic_North[nDive_slider.val]
    line_East.set_xdata(X1)
    dot_East.set_xdata(X1)
    line_North.set_xdata(X2)
    dot_North.set_xdata(X2)
    
    
    t_start = T[nDive_slider.val]
    if (nDive_slider.val==ADCP_DiveNumber[-1]):
        t_end = ADCP_time[-1]
    else :
        t_end = T[nDive_slider.val+1]
    date_start=datetime.datetime.utcfromtimestamp(t_start).strftime('%d/%b/%Y %H:%M' )
    date_end=datetime.datetime.utcfromtimestamp(t_end).strftime('%d/%b/%Y %H:%M' )
    
    ax3.clear()
    ax3.set_xlim3d(-0.8, 0.8)
    ax3.set_ylim3d(-0.8, 0.8)
    ax3.set_zlim3d(0, -1)
    ax3.set_xlabel('East speed [m/s]')
    ax3.set_ylabel('North speed [m/s]')
    plt.title(date_start + " to " + date_end)
    
    ax3.set_zlabel('\n \n Depth [m]')
    ax3.set_zticks(  np.linspace(0, -300, num=7)/300  )
    ax3.set_zticklabels(  np.linspace(0, -300, num=7)  )
    
    ax3.set_box_aspect((1, 1, 1))
    ax3.plot3D(zline*0, zline*0, zline/450, 'gray')
    ax3.quiver(zline*0, zline*0, zline/300, X1, X2, zline*0, arrow_length_ratio=0.15)
    ax3.quiver([0.4], [-0.4], [-1], [0], [0.5], [0], color='r', pivot='tail', length=1, arrow_length_ratio=0.3)
    plt.gca().invert_zaxis()
    plt.tight_layout()
    fig.canvas.draw_idle()

nDive_slider.on_changed(update)




fig2 = plt.figure()
ax3 = fig2.gca(projection='3d')
zline = np.linspace(-1, -H-1, num=Nz)
ax3.set_xlim3d(-0.8, 0.8)
ax3.set_ylim3d(-0.8, 0.8)
ax3.set_xlabel('East speed [m/s]')
ax3.set_ylabel('North speed [m/s]')

ax3.set_zlabel('\n \n Depth [m]')
ax3.set_zlim3d(0, -1)
ax3.set_zticks(  np.linspace(0, -300, num=7)/300    )
ax3.set_zticklabels(  np.linspace(0, -300, num=7)  )
#plt.gca().invert_zaxis()

ax3.set_box_aspect((1, 1, 1))
ax3.plot3D(zline*0, zline*0, zline/450, 'gray')
ax3.quiver(zline*0, zline*0, zline/300, X1, X2, zline*0, arrow_length_ratio=0.15)
ax3.quiver([0.4], [-0.4], [-1], [0], [0.5], [0], color='r', pivot='tail', length=1, arrow_length_ratio=0.3)
plt.tight_layout()

for angle in range(0, 360):
    ax3.view_init(30, angle)
    plt.draw()
    plt.show()
    plt.pause(.001)

fig.canvas.draw_idle()
fig2.canvas.draw_idle()
plt.show()

217
217


<ipython-input-68-779fa218f58f>:20: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  T=T[Selection]
<ipython-input-68-779fa218f58f>:21: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  X_East=X_East[Selection]
<ipython-input-68-779fa218f58f>:22: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  X_North=X_North[Selection]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-68-779fa218f58f>:133: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.tight_layout()


In [ ]:
#------------------------------------------------
#------------------ Create gif ------------------
#------------------------------------------------

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

ndive_to_show=78
X1= X_East[ndive_to_show] + Barotropic_East[ndive_to_show]
X2 = X_North[ndive_to_show] + Barotropic_North[ndive_to_show]
Y=np.linspace(-1, -H-1, num=Nz)



# Create a figure and a 3D Axes
fig = plt.figure()
ax = fig.gca(projection='3d')

def init():
    # Plot the speeds.
    t_start = T[nDive_slider.val]
    if (nDive_slider.val==ADCP_DiveNumber[-1]):
        t_end = ADCP_time[-1]
    else :
        t_end = T[nDive_slider.val+1]
    date_start=datetime.datetime.utcfromtimestamp(t_start).strftime('%d/%b/%Y %H:%M' )
    date_end=datetime.datetime.utcfromtimestamp(t_end).strftime('%d/%b/%Y %H:%M' )
    
    zline = np.linspace(-1, -H-1, num=Nz)
    ax.set_xlim3d(-0.8, 0.8)
    ax.set_ylim3d(-0.8, 0.8)
    ax.set_zlim3d(0, -1)
    ax.set_xlabel('East speed [m/s]')
    ax.set_ylabel('North speed [m/s]')
    ax.set_zlabel('\n \n Depth [m]')
    ax.set_zticks(  np.linspace(0, -300, num=7)/300   )
    ax.set_zticklabels(  np.linspace(0, -300, num=7)  )
    plt.title(date_start + " to " + date_end)
    ax.set_box_aspect((1, 1, 1))
    ax.plot3D(zline*0, zline*0, zline/450, 'gray')
    ax.quiver(zline*0, zline*0, zline/300, X1, X2, zline*0, arrow_length_ratio=0.15)
    ax.quiver([0.4], [-0.4], [-1], [0], [0.5], [0], color='r', pivot='tail', length=1, arrow_length_ratio=0.3)
    plt.gca().invert_zaxis()
    plt.tight_layout()
    return fig,

def animate(i):
    # azimuth angle : 0 deg to 360 deg
    ax.view_init(elev=30, azim=i)
    return fig,

# Animate
ani = animation.FuncAnimation(fig, animate, init_func=init, frames=360, interval=200, blit=True)

fn = '3d_current'
ani.save(fn+'.gif',writer='imagemagick',fps=1000/(50))

In [ ]:
#-----------------------------------------
#--------------- BROUILLON ---------------
#-----------------------------------------

#Animation ---------------- NOT TO USE 

import ipyvolume as ipv
import numpy as np
selected = np.random.randint(0, 1000, 100)
ipv.figure()
quiver = ipv.quiver(zline*0, zline*0, zline/100, X1, X2, zline*0, size=10, size_selected=8, selected=selected)

from ipywidgets import FloatSlider, ColorPicker, VBox, jslink
size = FloatSlider(min=0, max=30, step=0.1)
size_selected = FloatSlider(min=0, max=30, step=0.1)
color = ColorPicker()
color_selected = ColorPicker()
jslink((quiver, 'size'), (size, 'value'))
jslink((quiver, 'size_selected'), (size_selected, 'value'))
jslink((quiver, 'color'), (color, 'value'))
jslink((quiver, 'color_selected'), (color_selected, 'value'))
VBox([ipv.gcc(), size, size_selected, color, color_selected])

# PLOT THE TRAJECTORY

In [69]:
#Position with currents

NAV_position_East = np.zeros_like(flight.time)
NAV_position_North = np.zeros_like(flight.time)

for d in tqdm(range(ADCP_DiveNumber[-1]+1)):
    #Time interval to consider
    t_start = ADCP_DiveNumber_time[d]
    if (d==ADCP_DiveNumber[-1]):
        t_end = ADCP_time[-1]
    else :
        t_end = ADCP_DiveNumber_time[d+1] 
    delta_T = t_end-t_start
    dt = 1.0
    
    i1=(np.abs(NAV_time - t_start)).argmin()
    i2=(np.abs(NAV_time - t_end)).argmin()
    
    #Getting START and END GPS positions
    Start_latitude=np.interp(t_start, NAV_time, NAV_latitude)
    Start_longitude=np.interp(t_start, NAV_time, NAV_longitude)
    End_latitude=np.interp(t_end, NAV_time, NAV_latitude)
    End_longitude=np.interp(t_end, NAV_time, NAV_longitude)
    
    
    NAV_position_East[i1] = Start_longitude
    NAV_position_North[i1] = Start_latitude
    
    for i in np.arange( i1, i2-1):
        nz=NAV_nz[i]
        d_east_meters = (Glider_east_speed[i] + Baroclinic_East[d,nz] + Barotropic_East[d])*dt
        d_north_meters = (Glider_north_speed[i] + Baroclinic_North[d,nz] + Barotropic_North[d])*dt
        
        if (np.isnan(d_east_meters)):
            d_east_meters=0        
        if (np.isnan(d_north_meters)):
            d_north_meters=0
        
        NAV_position_East[i+1] = (distance.distance(meters=d_east_meters).destination((NAV_position_North[i], NAV_position_East[i]), bearing=90)).longitude
        NAV_position_North[i+1] = (distance.distance(meters=d_north_meters).destination((NAV_position_North[i], NAV_position_East[i]), bearing=0)).latitude
     
    NAV_position_East[i2] = End_longitude
    NAV_position_North[i2] = End_latitude

100%|████████████████████████████████████████████████████████████████████████████████| 217/217 [13:21<00:00,  3.69s/it]


In [70]:
#Position without currents

NAV_position_East_no_currents = np.zeros_like(flight.time)
NAV_position_North_no_currents = np.zeros_like(flight.time)

for d in tqdm(range(ADCP_DiveNumber[-1]+1)):
    #Time interval to consider
    t_start = ADCP_DiveNumber_time[d]
    if (d==ADCP_DiveNumber[-1]):
        t_end = ADCP_time[-1]
    else :
        t_end = ADCP_DiveNumber_time[d+1] 
    delta_T = t_end-t_start
    dt = 1.0
    
    i1=(np.abs(NAV_time - t_start)).argmin()
    i2=(np.abs(NAV_time - t_end)).argmin()
    
    #Getting START and END GPS positions
    Start_latitude=np.interp(t_start, NAV_time, NAV_latitude)
    Start_longitude=np.interp(t_start, NAV_time, NAV_longitude)
    End_latitude=np.interp(t_end, NAV_time, NAV_latitude)
    End_longitude=np.interp(t_end, NAV_time, NAV_longitude)
    
    
    NAV_position_East_no_currents[i1] = Start_longitude
    NAV_position_North_no_currents[i1] = Start_latitude
    
    for i in np.arange( i1, i2):
        nz=NAV_nz[i]
        d_east_meters = (Glider_east_speed[i])*dt
        d_north_meters = (Glider_north_speed[i])*dt
        
        if (np.isnan(d_east_meters)):
            d_east_meters=0        
        if (np.isnan(d_north_meters)):
            d_north_meters=0
        
        NAV_position_East_no_currents[i+1] = (distance.distance(meters=d_east_meters).destination((NAV_position_North_no_currents[i], NAV_position_East_no_currents[i]), bearing=90)).longitude
        NAV_position_North_no_currents[i+1] = (distance.distance(meters=d_north_meters).destination((NAV_position_North_no_currents[i], NAV_position_East_no_currents[i]), bearing=0)).latitude

100%|████████████████████████████████████████████████████████████████████████████████| 217/217 [13:30<00:00,  3.73s/it]


In [ ]:
#-----------------------------------------
#--------------- BROUILLON ---------------
#-----------------------------------------

d=105
t_start = ADCP_DiveNumber_time[d]
if (d==ADCP_DiveNumber[-1]):
    t_end = ADCP_time[-1]
else :
    t_end = ADCP_DiveNumber_time[d+1] 
delta_T = t_end-t_start

i1=(np.abs(NAV_time - t_start)).argmin()
i2=(np.abs(NAV_time - t_end)).argmin()


#Getting START and END GPS positions
Start_latitude=np.interp(t_start, NAV_time, NAV_latitude)
Start_longitude=np.interp(t_start, NAV_time, NAV_longitude)
End_latitude=np.interp(t_end, NAV_time, NAV_latitude)
End_longitude=np.interp(t_end, NAV_time, NAV_longitude)
    
    
NAV_position_East_no_currents[i1] = Start_longitude
NAV_position_North_no_currents[i1] = Start_latitude


A=np.zeros_like(np.arange( i1, i2))
B=np.zeros_like(np.arange( i1, i2))
for i in np.arange( i1, i2):
    nz=NAV_nz[i]
    d_east_meters = (Glider_east_speed[i])*dt
    d_north_meters = (Glider_north_speed[i])*dt
        
    if (np.isnan(d_east_meters)):
        d_east_meters=0        
    if (np.isnan(d_north_meters)):
        d_north_meters=0
        
    NAV_position_East_no_currents[i+1] = (distance.distance(meters=d_east_meters).destination((NAV_position_North[i], NAV_position_East[i]), bearing=90)).longitude
    NAV_position_North_no_currents[i+1] = (distance.distance(meters=d_north_meters).destination((NAV_position_North[i], NAV_position_East[i]), bearing=0)).latitude




print(NAV_position_North_no_currents[i1:i2])
print(NAV_position_North[i1:i2])

In [77]:
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

print(ADCP_DiveNumber[-1])

def plot_trajectory(d):
    #Time interval to consider
    t_start = ADCP_DiveNumber_time[d]
    if (d==ADCP_DiveNumber[-1]):
        t_end = ADCP_time[-1]
    else :
        t_end = ADCP_DiveNumber_time[d+1] 
    delta_T = t_end-t_start
    dt = 1.0
    
    i1=(np.abs(NAV_time - t_start)).argmin()
    i2=(np.abs(NAV_time - t_end)).argmin()
    
    #Getting START and END GPS positions
    Start_latitude=np.interp(t_start, NAV_time, NAV_latitude)
    Start_longitude=np.interp(t_start, NAV_time, NAV_longitude)
    End_latitude=np.interp(t_end, NAV_time, NAV_latitude)
    End_longitude=np.interp(t_end, NAV_time, NAV_longitude)
    
    #Figure plotting -----------------------------------------------------------------------------------
    %matplotlib widget
    plt.close('all')
    fig = plt.figure()
    #Plot vertical speed -----------------------------------------------------------------------------------------
    ax = fig.add_subplot(1, 1, 1)
    X=NAV_position_East[i1:i2]
    Y=NAV_position_North[i1:i2]
    X2=NAV_position_East_no_currents[i1:i2]
    Y2=NAV_position_North_no_currents[i1:i2]
    plt.plot(X,Y,label='Underwater trajectory (with currents)')
    plt.plot(X2,Y2,label='Underwater trajectory (no currents)', linestyle='--')
    plt.scatter([Start_longitude,End_longitude], [Start_latitude,End_latitude], label='GPS positions', marker='x', color='red', zorder=20)
    plt.legend()
    plt.grid()
    plt.xlabel("Longitude [°]")
    plt.ylabel("Latitude [°]")
    fig.canvas.draw_idle()
    if (  (data.data['Timestamp'][i1]).strftime('%d %B %Y')==(data.data['Timestamp'][i2]).strftime('%d %B %Y')) :
        plt.title(  (data.data['Timestamp'][i1]).strftime('%d %b %Y - %H:%M') + ' to ' + (data.data['Timestamp'][i2]).strftime('%H:%M  '))
    else :
        plt.title(  (data.data['Timestamp'][i1]).strftime('%d %b %Y - %H:%M') + ' - ' +   (data.data['Timestamp'][i2]).strftime('%d %b %Y - %H:%M'))
    plt.tight_layout()
    ax.set_aspect(1/np.cos( np.deg2rad((Start_latitude+End_latitude)/2) )) #Build a local mercator canvas
    scalebar = AnchoredSizeBar(ax.transData,
                           (1/(60*1.852)), '1 km', 'lower center', 
                           pad=0.01,
                           color='black',
                           frameon=False,
                           size_vertical=0.0001)   
    ax.add_artist(scalebar)
    plt.show()

plot_trajectory(139)   

216


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# PLOT OF THE DIFFERENT RESULTS 

### Plot Bottom tracking data

In [180]:
x=46

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()



X2=pd.DataFrame( np.asarray(BT.time/(10**3)).T,columns=['Date'])
X2['Date']=pd.to_datetime(X2['Date'], unit='us', utc=True, origin='unix', cache='False')
X2['Timestamp']=X2['Date']
X2.set_index('Date', inplace= True)
X2=X2['Timestamp']
#Y5=-np.asarray(BT.isel(bin=0)['U'])
Y5=np.asarray(BT_spd_xyz)
print(len(X2))



#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()


ax = fig.add_subplot(1, 1, 1)
X=data.data['Timestamp'][i1:i2]
Y1 = ADCP_speed[i1:i2]
Y2 = dZdt[i1:i2]
Y3 = -np.gradient(ADCP_pressure,time)[i1:i2]
Y4 = -BT_vert_spd[i1:i2]
plt.plot(X, Y1, alpha=0.5, label='ADCP')
#plt.plot(X, Y2, alpha=0.5, label='Pressure sensor ')
#plt.plot(X, Y3, alpha=0.7, label='ADCP pressure sensor UP speed')
#plt.plot(X, Y4, alpha=1, label='BT UP speed')





C1=np.asarray(BT['FoM1'].copy())
C2=np.asarray(BT['FoM2'].copy())
C3=np.asarray(BT['FoM3'].copy())
C4=np.asarray(BT['FoM4'].copy())
C1=C1.astype('float')
C2=C2.astype('float')
C3=C3.astype('float')
C4=C4.astype('float')
C=np.asarray(BT['D1'])
#C=np.nan_to_num(C)
plt.scatter(X2, Y5, 1, alpha=1, color='r', label='Bottom Tracking',zorder=10)

ax.set_ylim([0,0.4])
ax.set_xlim([X[0],X[-1]])


plt.ylabel('speed [m/s]')
if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
plt.title('Vertical speed')


plt.grid()
plt.legend()
xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
fig.canvas.draw_idle()

plt.show()

33435


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
print(np.nanmean(   np.abs(flight.alpha[flight._valid])   ))
print(np.nanmean(   np.abs(flight.pitch[flight._valid])   ))
print(np.nanmean(   np.abs(flight.alpha[flight._valid] + flight.pitch[flight._valid])   ))
print(np.nanmean(   np.abs((flight.F_B[flight._valid] -flight.F_g[flight._valid])*np.sin(flight.alpha[flight._valid] + flight.pitch[flight._valid]))   ))
print(np.nanmean(   np.abs((flight.F_B[flight._valid] -flight.F_g[flight._valid])*np.sin( flight.pitch[flight._valid]))   ))


print(  0.0465087374050051 + 1.27273918009592*(0.07260724645085814**2)  )
print(np.nanmean(   np.abs(flight.rho[flight._valid])   ))

TAU=(60.772+3.46)*np.sqrt(  2/(1027.397838710252*0.24*0.05321837938936768*2*0.9970374456177116)  )
print("tau = " + str(TAU) + "s")
print(np.nanmean(   90*np.abs(np.sin(flight.alpha[flight._valid] ))))

In [84]:
x=3

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/10
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()

In [85]:
x=30 #291

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()




#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()





#Plot vertical speed -----------------------------------------------------------------------------------------
ax = fig.add_subplot(2, 2, 1)

X=data.data['Timestamp'][i1:i2]
Y1=dZdt[i1:i2]
Y2=data.data.speed_vert[i1:i2]
Y3=flight.speed_vert[i1:i2]


plt.plot(X,Y1,label='Pressure sensor')
plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Vertical glider speed")
plt.ylabel("speed [m/s]")
ax.set_ylim([-0.4,0.8])



#Plot total speed -----------------------------------------------------------------------------------------
ax2 = fig.add_subplot(2, 2, 2)

X=data.data['Timestamp'][i1:i2]
Y1=ADCP_speed[i1:i2]
Y2=data.data.speed[i1:i2]
Y3=flight.speed[i1:i2]

plt.plot(X,Y1,label='ADCP')
plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Total glider speed")
plt.ylabel("speed [m/s]")
ax2.set_ylim([-0.1,1])



x=306.7 #291

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/1000
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()

#Plot vertical speed -----------------------------------------------------------------------------------------
ax3 = fig.add_subplot(2, 2, 3)

X=data.data['Timestamp'][i1:i2]
Y1=dZdt[i1:i2]
Y2=data.data.speed_vert[i1:i2]
Y3=flight.speed_vert[i1:i2]


plt.plot(X,Y1,label='Pressure sensor')
plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Vertical glider speed")
plt.ylabel("speed [m/s]")
ax3.set_ylim([-0.4,0.8])



#Plot total speed -----------------------------------------------------------------------------------------
ax4 = fig.add_subplot(2, 2, 4)

X=data.data['Timestamp'][i1:i2]
Y1=ADCP_speed[i1:i2]
Y2=data.data.speed[i1:i2]
Y3=flight.speed[i1:i2]

plt.plot(X,Y1,label='ADCP')
plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Total glider speed")
plt.ylabel("speed [m/s]")
ax4.set_ylim([-0.1,1])




fig.canvas.draw_idle()

if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    fig.suptitle(X[0].strftime('%d %B %Y'))
else :
    fig.suptitle(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
    
#fig.suptitle('This is a somewhat long figure title', fontsize=16)

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[2].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[3].xaxis.set_major_formatter(xformatter)
#plt.tight_layout()
plt.show()


print(i1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

847106


In [86]:
x=30

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()




#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()





#Plot vertical speed -----------------------------------------------------------------------------------------
ax = fig.add_subplot(2, 2, 1)

X=data.data['Timestamp'][i1:i2]
Y1=dZdt[i1:i2]
Y2=data.data.speed_vert[i1:i2]
Y3=flight.speed_vert[i1:i2]


plt.plot(X,Y1,label='Pressure sensor')
#plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Vertical glider speed")
plt.ylabel("speed [m/s]")
ax.set_ylim([-0.4,0.8])



#Plot total speed -----------------------------------------------------------------------------------------
ax2 = fig.add_subplot(2, 2, 2)

X=data.data['Timestamp'][i1:i2]
Y1=smooth(ADCP_speed,50)[i1:i2]
Y2=data.data.speed[i1:i2]
Y3=flight.speed_filtered[i1:i2]

plt.plot(X,Y1,label='ADCP')
#plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Total glider speed")
ax2.set_ylim([-0.4,0.8])




#Plot vertical speed  error -----------------------------------------------------------------------------------------
ax3 = fig.add_subplot(2, 2, 3)

X=data.data['Timestamp'][i1:i2]
Y=dZdt[i1:i2]-flight.speed_vert[i1:i2]
C=np.abs(   np.gradient(flight.speed_vert[i1:i2],flight.time[i1:i2])   )

plt.scatter(X,Y,3,c=C,cmap='plasma_r', alpha=1)
plt.clim([0,0.04])

cbar = plt.colorbar(orientation="horizontal", pad=0.2)
cbar.ax.set_title('|az| [m2/s]', rotation=0)

plt.grid()
plt.title("Vertical speed error")
plt.ylabel("speed error [m/s]")
ax3.set_ylim([-0.3,0.42])



#Plot total speed  error -----------------------------------------------------------------------------------------
ax4 = fig.add_subplot(2, 2, 4)

X=data.data['Timestamp'][i1:i2]
Y=smooth(ADCP_speed, 50)[i1:i2]-flight.speed[i1:i2]
C=np.abs(   np.gradient(flight.speed[i1:i2],flight.time[i1:i2])   )

plt.scatter(X,Y,3,c=C,cmap='plasma_r', alpha=1)
plt.clim([0,0.08])

cbar = plt.colorbar(orientation="horizontal", pad=0.2)
cbar.ax.set_title('|a| [m2/s]', rotation=0)

plt.grid()
plt.title("Total speed error")
ax4.set_ylim([-0.3,0.42])




fig.canvas.draw_idle()

if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    fig.suptitle(X[0].strftime('%d %B %Y'))
else :
    fig.suptitle(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
    
#fig.suptitle('This is a somewhat long figure title', fontsize=16)

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[2].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[4].xaxis.set_major_formatter(xformatter)  # /!\ les cbar sont considérées comme des axes x
#plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
x=30


t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()




#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()




#Plot total speed -----------------------------------------------------------------------------------------
ax2 = fig.add_subplot(2, 1, 1)

X=data.data['Timestamp'][i1:i2]
Y1=ADCP_speed[i1:i2]
Y2=data.data.speed[i1:i2]
Y3=flight.speed[i1:i2]

plt.plot(X,Y1,label='ADCP')
#plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')
plt.ylabel("speed [m/s]")

plt.legend()
plt.grid()
plt.title("Total glider speed")
ax2.set_ylim([0,0.8])







#-----------------------------------------------------------------------------------------
ax = fig.add_subplot(2, 1, 2)

X=data.data['Timestamp'][i1:i2]
Y=np.gradient(flight.speed[i1:i2],flight.time[i1:i2])

V=np.nanmean(flight.speed)
L=2.0
a_max = V*V/(10*L)

plt.scatter(X,Y, 1, color='red', linestyle='dashed', label='Steady state acceleration')
plt.plot([X[0],X[-1]],[a_max,a_max], '--k')
plt.plot([X[0],X[-1]],[-a_max,-a_max], '--k', label='Validity domain')

plt.legend()
plt.grid()
plt.ylabel("acceleration [m2/s]")
plt.title("Acceleration criteria")
#ax.set_ylim([-0.08,0.11])

if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)
#plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fast Fourier Transform of ADCP signal

In [ ]:
from scipy.fft import fft, fftfreq

#Compute average dive time for the entire flight ---------------------------------------------------------------
diveNum= np.asarray(  data.data['diveNum'].values  )
diveTime= np.asarray(  sx.date2float(data.data['Timestamp']).values  )
index=np.zeros( (diveNum[-1],) , dtype=int)
for i in range(diveNum[-1]) :
    index[i] = np.argmax(diveNum==i+1)
for i in range(diveNum[-1]) :
    index[i] = int(diveTime[index[i]])
T_dive=np.mean(np.diff(index))
print("Average dive period [s] : ")
print(T_dive)
f_dive=1/T_dive
print("Average dive frequency [Hz] : ")
print(f_dive)


#Compute average dive time for the interval considered ---------------------------------------------------------------

x=30#Use this variable to chose the interval

time=sx.date2float(data.data['Timestamp'])
t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()

diveNum= np.asarray(  data.data['diveNum'].values  )[i1:i2]
diveTime= np.asarray(  sx.date2float(data.data['Timestamp']).values  )[i1:i2]
index=np.zeros( (diveNum[-1]-diveNum[0],) , dtype=int)
for i in range(diveNum[-1]-diveNum[0]) :
    index[i] = np.argmax(diveNum==i+diveNum[0])
for i in range(diveNum[-1]-diveNum[0]) :
    index[i] = int(diveTime[index[i]])
T_dive=np.mean(np.diff(index))
print("Average dive period [s] : ")
print(T_dive)
f_dive=1/T_dive
print("Average dive frequency [Hz] : ")
print(f_dive)
#----------------------------------------------------------------------------------------------------------------------



#Compute FFT -------------------------------------------------------------------------------------------------------------
ADCP_Time=ADCP.time.values.astype('float')/(10**9)
ADCP_spd_xyz = np.asarray(  np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Y']**2 + ADCP.isel(bin=0)['Z']**2)  )


#Interval to study
t = np.nanpercentile(ADCP_Time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(ADCP_Time - x1)).argmin()
i2=(np.abs(ADCP_Time - x2)).argmin()



#Raw data
N = len(ADCP_Time[i1:i2])
T = 10
x = ADCP_Time[i1:i2]
y = ADCP_spd_xyz[i1:i2]


#smooth data
#y=smooth(y,3)

#Delete the last data because absurd result in time
#N = len(ADCP_Time[0:-10])
#T = 10
#x = ADCP_Time[0:-10]
#y = ADCP_spd_xyz[0:-10]


#Delete the constant part of speed
print("Average ADCP speed [m/s] :")
print(np.mean(y))
y=y-np.mean(y)

#Sampled data
#T = 50
#x=x[::5]
#y=y[::5]
#N = len(x)


yf = fft(y)
xf = fftfreq(N, T)[:N//2]

#Plotting figure ------------------------------------------------------------------------
plt.close('all')
fig = plt.figure()


ax0 = fig.add_subplot(2, 1, 1)
T=pd.to_datetime(x, unit='s', utc=True, origin='unix', cache='False')
plt.plot(T, y)
plt.title('(ADCP speed) - (ADCP average speed)')
if (T[0].strftime('%d %B %Y')==T[-1].strftime('%d %B %Y')) :
    plt.xlabel(T[0].strftime('%d %B %Y'))
else :
    plt.xlabel(T[0].strftime('%d %B %Y') + ' - ' + T[-1].strftime('%d %B %Y'))
plt.ylabel('Speed [m/s]')
xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.grid()

ax = fig.add_subplot(2, 1, 2)
plt.plot(xf, 2.0/N*np.abs(yf[0:N//2]), label="FFT(ADCP speed)")
plt.plot([f_dive,f_dive], [-1,1], label="Average dive frequency")
plt.xlabel('Frequency [Hz]')
plt.title('FFT(ADCP speed - ADCP average speed)')
ax.set_ylim([0,0.06])
ax.set_xscale('log')
#ax.set_xlim([0,0.01])
plt.grid()


plt.legend()
plt.tight_layout()
plt.show()

# Searching for errors

### Computation of useful variables for plotting (ex: spd error between Steady state VS ADCP)

# STEADY STATE 1

In [ ]:
importlib.reload(sx)
    
def flight_model():              
    flight = sx.SlocumModel(
               data.data.Timestamp, 
               data.data.salinity.interpolate('index').values, 
               data.data.temperature.interpolate('index').values, 
               data.data.pressure.interpolate('index').values, 
               data.data.lon.interpolate('index').values, 
               data.data.lat.interpolate('index').values, 
               data.data.BallastPos.interpolate('index').values, 
               ADCP_pitch, 
               data.data.profileNum.interpolate('nearest').values,
               data.data.NAV_RESOURCE.interpolate('nearest').values,
               mass=60.772) # Verify mass number !!  
    flight.model_function()
    data.data['speed_vert_notRegressed'] = flight.speed_vert   
    flight.regress()     
    data.data['alpha'] = flight.alpha
    data.data['speed'] = flight.speed
    data.data['speed_vert'] = flight.speed_vert
    data.data['speed_horz'] = flight.speed_horz
    data.data['w_H2O'] = flight.w_H2O
    data.data['_valid'] = flight._valid     
    return flight
       
flight_old = flight_model()

In [ ]:
aoa=fillGaps(time, data.data.alpha)
Pitch=fillGaps(time, np.deg2rad(data.data.Pitch)) #DO NOT USE !!! Data is wrong when mass is moving
Roll=fillGaps(time, np.deg2rad(data.data.Roll))

speed=data.data.speed.values.astype('float')
vertical_speed =fillGaps(time, data.data.speed_vert)
vertical_speed_notRegressed =fillGaps(time, data.data.speed_vert_notRegressed)
horizontal_speed = speed*np.cos(aoa+Pitch)

#Acceleration criteria validity --------------------------------------------------------------------------------------
L = 2.7 #length of glider
V = np.nanmean(data.data.speed) #Average speed of drone
a_max = V*V/(10*L)
a = np.gradient(speed,time)

validity = np.sign(np.abs(a)-a_max)

#Speed error SS1
spd_err=ADCP_speed-speed

In [ ]:
%matplotlib widget
plt.close('all')

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.plot(data.data['Timestamp'].values.astype('float'), data.data.speed,'--k', alpha=0.5,label="steady flight model")
plt.plot(data.data['Timestamp'].values.astype('float'),ADCP_speed, alpha=0.7, color='g',label="ADCP")
plt.title("speed of glider")
plt.xlabel("t [s]")
plt.ylabel("v [m/s]")
plt.legend()

t = np.nanpercentile(data.data['Timestamp'].values.astype('float'),[0,100])


def update(x=0):
    ax.set_xlim([t[0] + x*step , t[0] + x*step + step])
    fig.canvas.draw_idle()

ax.set_ylim([0,1.2])
step = np.diff(t)/100
update(30)


print((t[0] + 30*step)/10**9 )
print(t[0] + 30*step + step)
print(np.interp(1.59846038e+18, data.data['Timestamp'].values.astype('float'),  data.data.speed))
print(1.59847874e+18-1.59846038e+18)



plt.show()

#### Plotting influence of different parameters (Temperature, Pressure, Salinity, Pitch, Alpha, ...)

In [ ]:
#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

X=data.data['Timestamp'][i1:i2]
#Y1=dZdt[i1:i2]
Y1=(Depth)[i1:i2]
Y2=data.data.speed_vert_notRegressed[i1:i2]
Y3=data.data.speed_vert[i1:i2]


#X=X[data.data.Depth[i1:i2]<-10]
#Y1=Y1[data.data.Depth[i1:i2]<-10]
#Y2=Y2[data.data.Depth[i1:i2]<-10]
#Y3=Y3[data.data.Depth[i1:i2]<-10]

plt.plot(X,Y1,label='Pressure sensor')
#plt.plot(X,Y3,label='Steady state (regressed R1)',linestyle='--')
#plt.plot(X,Y2,label='Steady state (not regressed)',linestyle='--',color='brown')
plt.grid()
plt.title("Vertical glider speed")
plt.legend()
if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
plt.ylabel("speed [m/s]")

#ax.set_ylim([-1,1])
fig.canvas.draw_idle()


xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)


plt.show()

In [ ]:
#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
#plt.scatter(data.data['Timestamp'].values.astype('float'),spd_err, 1,c=data.data.Temperature,cmap='coolwarm', alpha=1)
#plt.scatter(time[_u],spd_err[_u], 1,'b', alpha=1,label='upcast')
X=data.data['Timestamp'][i1:i2]
Y=spd_err[i1:i2]
C=a[i1:i2]
#X=X[Depth[i1:i2]<-10]
#Y=Y[Depth[i1:i2]<-10]
#C=C[Depth[i1:i2]<-10]


plt.scatter(X,Y, 1,c=C,cmap='coolwarm', alpha=1, label='Speed error')
cbar = plt.colorbar()
cbar.set_label('acceleration [m2/s]')
plt.clim([-0.0005,0.0005])
plt.grid()
plt.title("speed error (for depth > 1m)")
if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
plt.ylabel("speed error [m/s]")
plt.legend()
ax.set_ylim([-0.8,0.8])
fig.canvas.draw_idle()


xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)

plt.show()

# Sensor Comparison

#### Vertical Speed

In [ ]:
#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
X=data.data['Timestamp'][i1:i2]
Y1=smooth(-ADCP_vert_spd,1)[i1:i2]
Y2 = dZdt[i1:i2]
plt.plot(X, Y1, alpha=0.7, label='ADCP UP speed')
plt.plot(X, Y2, alpha=0.7, label='Pressure sensor UP speed')
plt.grid()
plt.legend()
plt.title("Vertical speed sensor comparison")
if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
plt.ylabel("Vertical speed [m/s]")


fig.canvas.draw_idle()
ax.set_ylim([-0.4,0.6])

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)

plt.show()

#### Horizontal Speed

In [ ]:
#       TO DO !!! (insert bottom tracking data here)

# Dynamic flight model -------------------------------------------------------------------------

In [ ]:
import dynamic_model as DM
importlib.reload(DM)

flight_dyn = DM.DynamicFlightModel(
               data.data.Timestamp, 
               data.data.salinity.interpolate('index').values, 
               data.data.temperature.interpolate('index').values, 
               data.data.pressure.interpolate('index').values, 
               data.data.lon.interpolate('index').values, 
               data.data.lat.interpolate('index').values, 
               data.data.BallastPos.interpolate('index').values, 
               ADCP_pitch, 
               data.data.profileNum.interpolate('nearest').values,
               data.data.NAV_RESOURCE.interpolate('nearest').values,
               ADCP_speed)


flight_dyn.set_initial_conditions(flight.speed_horz[558109],flight.speed_vert[558109])
flight_dyn.solveRK4()

In [ ]:
flight_dyn.regress()

#### Comparison of  TOTAL SPEED (Dyanmic VS Static VS ADCP)

In [ ]:
x=306.7 #291

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/1000
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()



X=data.data['Timestamp'][i1:i2]


ADCP_alpha=np.arctan2(-ADCP_vert_spd,-ADCP_hrz_spd)-ADCP_pitch



#Dynamic results

Dyn_Speed= np.sqrt(flight_dyn.w**2+flight_dyn.u**2)
Dyn_Speed=np.interp(time, flight_dyn.t, Dyn_Speed)
Dyn_Speed=Dyn_Speed[i1:i2]

Dyn_Speed_vert=np.interp(time, flight_dyn.t, flight_dyn.w)
Dyn_Speed_vert=Dyn_Speed_vert[i1:i2]

Dyn_alpha=np.arctan2(flight_dyn.w,flight_dyn.u)-np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch)
Dyn_alpha=np.interp(time, flight_dyn.t, Dyn_alpha)
Dyn_alpha=Dyn_alpha[i1:i2]

#Fb, Fg = flight_dyn.compute_FB_and_Fg(g, rho, pressure, ballast, temperature)
#for i in range(len(flight_dyn.t)) :
#    L, D=flight_dyn.compute_lift_and_drag(pitch[i], rho[i], flight_dyn.u[i], flight_dyn.w[i])
#L, D=flight_dyn.compute_lift_and_drag(pitch, rho, flight_dyn.u, flight_dyn.w)
#Fx=np.sin(pitch + alpha)*L-np.cos(pitch + alpha)*D
#Fy=Fb - Fg -np.cos(pitch + alpha)*L -np.sin(pitch + alpha)*D





#Plotting figures
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(2, 1, 1)

#ADCP speed
#plt.plot(time, -ADCP_vert_spd, 'b',label="w ADCP [m/s]") #ADCP speed interpolated 
#plt.plot(time, ADCP_x_spd, 'b',label="w ADCP [m/s]") #ADCP speed interpolated 
#plt.plot(time,-ADCP_hrz_spd,'b', alpha=1, label='ADCP speed') #ADCP speed not interpolated 
plt.plot(X,ADCP_speed[i1:i2], alpha=1, label='ADCP') #ADCP speed not interpolated 

#Pressure sensor
#plt.plot(X, dZdt[i1:i2], 'm',label="w Pressure sensor [m/s]")

#Dynamic model forces
#plt.plot(flight_dyn.t, ballast*1000, 'k')
#plt.plot(flight_dyn.t, pitch/1.9, '--k')
#plt.plot(flight_dyn.t, D/10, 'y')
#plt.plot(flight_dyn.t, L/10, 'y')

#Dynamic model pitch, depth, alpha, ...
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch), 'g')
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch), 'g')
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.depth), 'k')
#plt.plot(flight_dyn.t, (np.arctan2(flight_dyn.w,flight_dyn.u)-np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch)), '--r')


#Steady state speed
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, DM.date2float(data.data.Timestamp), data.data.speed_horz),'--y',label="u stat [m/s]")
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, DM.date2float(data.data.Timestamp), flight.speed_vert),'--y',label="w steady state [m/s]")
#plt.plot(X, flight.speed[i1:i2], '--',label="Steady state (R2)")


#Dynamic model Speed
#plt.plot(flight_dyn.t, flight_dyn.u, 'r', label="u dynamic [m/s]")
#plt.plot(flight_dyn.t, flight_dyn.w, 'r', label="w dynamic [m/s]")
#plt.plot(flight_dyn.t, np.sqrt(flight_dyn.w**2+flight_dyn.u**2), color='red', label="U dynamic model")
plt.plot(X, Dyn_Speed, color='red', label="Dynamic model")


#SEMI Steady state speed
plt.plot(X, flight.speed_filtered[i1:i2], ':k',label="Semi steady state (R2)")

plt.grid()
plt.ylabel("Speed [m/s]")
plt.title("Total speed")
plt.legend()
ax.set_ylim([0,0.8])






ax2 = fig.add_subplot(2, 1, 2)

plt.plot(X, dZdt[i1:i2],label="Pressure sensor")
#plt.plot(X, flight.speed_vert[i1:i2], '--',label="Steady state (R2)")
plt.plot(X, Dyn_Speed_vert, color='red', label="Dynamic model")
plt.plot(X, flight.speed_vert_filtered[i1:i2], linestyle='--',color='k',alpha=0.6,label="Semi steady state (R2)")

#plt.plot(X, flight.alpha[i1:i2], 'm',label="Semi steady state (R2)")
#plt.plot(X, flight.vert_dir[i1:i2], 'y',label="Semi steady state (R2)")
#plt.plot(X, flight.ballast[i1:i2]*1000, 'b',label="Ballast")
#plt.plot(X, flight.pitch[i1:i2], 'y',label="Semi steady state (R2)")
#plt.plot(X, Dyn_alpha, 'y',label="Semi steady state (R2)")
#plt.plot(X, Pitch[i1:i2], 'y',label="Pitch NAV")
#plt.plot(X, ADCP_pitch[i1:i2], ':r',label="Pitch ADCP")

plt.grid()
plt.ylabel("Speed [m/s]")
plt.title("Vertical speed ")
plt.legend()
ax.set_ylim([0,0.8])

if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))


print(X.values.astype('float')[0])

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)
plt.tight_layout()
plt.show()

#### Comparison of speed error (relative to ADCP) between dynamic and static

In [ ]:
x=306.7 #291

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/1000
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()



# Definition of error function
Dyn_Speed= np.sqrt(flight_dyn.w**2+flight_dyn.u**2)

Dyn_spd_err = ADCP_speed - np.interp(time, flight_dyn.t, Dyn_Speed)
Dyn_spd_vert_err = dZdt - np.interp(time, flight_dyn.t, flight_dyn.w)
Dyn_spd_err=Dyn_spd_err[i1:i2]
Dyn_spd_vert_err=Dyn_spd_vert_err[i1:i2]

Stead_spd_err =  (ADCP_speed-flight.speed)[i1:i2]
Stead_spd_vert_err = (dZdt-flight.speed_vert)[i1:i2]
X=data.data['Timestamp'][i1:i2]



#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()


ax = fig.add_subplot(2, 1, 1)

plt.plot(X, Dyn_spd_err, 'r', alpha=1, label ='Dynamic model')
plt.plot(X, Stead_spd_err, '--',color='orange', alpha=1, label ='Steady state model')

plt.grid()
plt.legend()
plt.title("Total speed error")
plt.ylabel("speed error [m/s]")
ax.set_ylim([-0.3,0.3])



ax2 = fig.add_subplot(2, 1, 2)

plt.plot(X, Dyn_spd_vert_err, 'r', alpha=1, label ='Dynamic model')
plt.plot(X, Stead_spd_vert_err, '--',color='orange', alpha=1, label ='Steady state model')

plt.grid()
plt.legend()
plt.title("Vertical speed error")
plt.ylabel("speed error [m/s]")
ax2.set_ylim([-0.3,0.5])



fig.canvas.draw_idle()

if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)
plt.tight_layout()
plt.show()

#### Comparison of  HORIZONTAL SPEED (Dyanmic VS Static VS ADCP)